<a href="https://colab.research.google.com/github/Awonke03/Amazon-Stock-Dashboard/blob/main/Amazon_DashBoard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:


!pip install dash


In [18]:
import dash
from dash import dcc, html
import yfinance as yf
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

app = dash.Dash(__name__)

# Function to get Amazon stock data
def get_amazon_stock_data():
    amazon_data = yf.download("AMZN", start="2020-01-01", end="2023-01-01")
    return amazon_data

# Function to train and predict using a linear regression model
def predict_stock_prices(data):
    # Features for prediction
    features = ['Open', 'High', 'Low', 'Close', 'Volume']

    # Add lagged features for time series prediction
    for feature in features:
        data[f'{feature}_lag1'] = data[feature].shift(1)

    # Drop NaN values
    data = data.dropna()

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(data[features], data['Close'], test_size=0.2, random_state=42)

    # Initialize and train the model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Make predictions
    predictions = model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, predictions)
    print(f'Mean Squared Error: {mse}')


    return predictions

# Function for MACD Calculation
def calculate_macd(data):
    data['12D_EMA'] = data['Close'].ewm(span=12, adjust=False).mean()
    data['26D_EMA'] = data['Close'].ewm(span=26, adjust=False).mean()
    data['MACD'] = data['12D_EMA'] - data['26D_EMA']
    data['Signal_Line'] = data['MACD'].ewm(span=9, adjust=False).mean()
    return data

# Function for Bollinger Bands Calculation
def calculate_bollinger_bands(data):
    data['SMA20'] = data['Close'].rolling(window=20).mean()
    data['Upper_Band'] = data['SMA20'] + 2 * data['Close'].rolling(window=20).std()
    data['Lower_Band'] = data['SMA20'] - 2 * data['Close'].rolling(window=20).std()
    return data

# Function for RSI Calculation
def calculate_rsi(data, window=14):
    close_price = data['Close']
    daily_returns = close_price.pct_change().dropna()
    gain = daily_returns.apply(lambda x: x if x > 0 else 0)
    loss = daily_returns.apply(lambda x: -x if x < 0 else 0)
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    rs = avg_gain / avg_loss
    data['RSI'] = 100 - (100 / (1 + rs))
    return data

# Get Amazon stock data
amazon_stock_data = get_amazon_stock_data()

# Calculate MACD, Bollinger Bands, and RSI
amazon_stock_data = calculate_macd(amazon_stock_data)
amazon_stock_data = calculate_bollinger_bands(amazon_stock_data)
amazon_stock_data = calculate_rsi(amazon_stock_data)

# Get predictions using the trained model
predictions = predict_stock_prices(amazon_stock_data)

# Define layout
app.layout = html.Div([
    html.H1("Amazon Stock Analysis Dashboard"),

    # Line Chart - Stock Price Trend
    dcc.Graph(
        id='stock-trend',
        figure={
            'data': [
                go.Scatter(
                    x=amazon_stock_data.index,
                    y=amazon_stock_data['Close'],
                    mode='lines',
                    name='Amazon Stock Price',
                ),
            ],
            'layout': go.Layout(
                title='Amazon Stock Price Trend',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Stock Price (USD)'},
                hovermode='closest',
            ),
        },
    ),

    # Candlestick Chart
    dcc.Graph(
        id='candlestick-chart',
        figure={
            'data': [
                go.Candlestick(
                    x=amazon_stock_data.index,
                    open=amazon_stock_data['Open'],
                    high=amazon_stock_data['High'],
                    low=amazon_stock_data['Low'],
                    close=amazon_stock_data['Close'],
                    name='Amazon Stock Prices',
                ),
            ],
            'layout': go.Layout(
                title='Amazon Stock Prices (Candlestick)',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Stock Price (USD)'},
                hovermode='closest',
            ),
        },
    ),

    # Moving Average
    dcc.Graph(
        id='moving-average',
        figure={
            'data': [
                go.Scatter(
                    x=amazon_stock_data.index,
                    y=amazon_stock_data['Close'],
                    mode='lines',
                    name='Amazon Stock Price',
                ),
                go.Scatter(
                    x=amazon_stock_data.index,
                    y=amazon_stock_data['Close'].rolling(window=20).mean(),
                    mode='lines',
                    name='20-Day Moving Average',
                ),
            ],
            'layout': go.Layout(
                title='Amazon Stock Price with 20-Day Moving Average',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Stock Price (USD)'},
                hovermode='closest',
            ),
        },
    ),

    # Daily Returns
    dcc.Graph(
        id='daily-returns',
        figure={
            'data': [
                go.Bar(
                    x=amazon_stock_data.index,
                    y=amazon_stock_data['Close'].pct_change(),
                    name='Daily Returns',
                ),
            ],
            'layout': go.Layout(
                title='Amazon Daily Returns',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Percentage Change'},
                hovermode='closest',
            ),
        },
    ),

    # Volume Chart
    dcc.Graph(
        id='volume-chart',
        figure={
            'data': [
                go.Bar(
                    x=amazon_stock_data.index,
                    y=amazon_stock_data['Volume'],
                    name='Trading Volume',
                ),
            ],
            'layout': go.Layout(
                title='Amazon Trading Volume',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Volume'},
                hovermode='closest',
            ),
        },
    ),

    # Historical Highs and Lows
    dcc.Graph(
        id='high-low-chart',
        figure={
            'data': [
                go.Scatter(
                    x=amazon_stock_data.index,
                    y=amazon_stock_data['High'],
                    mode='lines',
                    name='High Price',
                ),
                go.Scatter(
                    x=amazon_stock_data.index,
                    y=amazon_stock_data['Low'],
                    mode='lines',
                    name='Low Price',
                ),
            ],
            'layout': go.Layout(
                title='Amazon Historical Highs and Lows',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Stock Price (USD)'},
                hovermode='closest',
            ),
        },
    ),

    # Rolling Standard Deviation
    dcc.Graph(
        id='rolling-std',
        figure={
            'data': [
                go.Scatter(
                    x=amazon_stock_data.index,
                    y=amazon_stock_data['Close'].rolling(window=20).std(),
                    mode='lines',
                    name='20-Day Rolling Standard Deviation',
                ),
            ],
            'layout': go.Layout(
                title='Amazon 20-Day Rolling Standard Deviation',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Standard Deviation'},
                hovermode='closest',
            ),
        },
    ),

    # Monthly Average Closing Price
    dcc.Graph(
        id='monthly-average',
        figure={
            'data': [
                go.Bar(
                    x=amazon_stock_data.resample('M').mean().index,
                    y=amazon_stock_data.resample('M').mean()['Close'],
                    name='Monthly Average Closing Price',
                ),
            ],
            'layout': go.Layout(
                title='Amazon Monthly Average Closing Price',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Stock Price (USD)'},
                hovermode='closest',
            ),
        },
    ),

    # Rolling Correlation with S&P 500
    dcc.Graph(
        id='rolling-correlation',
        figure={
            'data': [
                go.Scatter(
                    x=amazon_stock_data.index,
                    y=amazon_stock_data['Close'].pct_change().rolling(window=20).corr(),
                    mode='lines',
                    name='Rolling Correlation with S&P 500',
                ),
            ],
            'layout': go.Layout(
                title='Amazon Rolling Correlation with S&P 500',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Correlation'},
                hovermode='closest',
            ),
        },
    ),

    # Sunburst Chart
    dcc.Graph(
        id='sunburst-chart',
        figure={
            'data': [
                go.Sunburst(
                    labels=["Amazon", "Stock Analysis", "Dashboard"],
                    parents=["", "Amazon", "Stock Analysis"],
                    values=[10, 5, 1],
                ),
            ],
            'layout': go.Layout(
                title='Sunburst Chart - Dashboard Hierarchy',
                hovermode='closest',
            ),
        },
    ),

    # Scatter Mapbox Chart
    dcc.Graph(
        id='scatter-mapbox',
        figure={
            'data': [
                go.Scattermapbox(
                    lat=[37.7749],
                    lon=[-122.4194],
                    mode='markers',
                    marker=dict(size=14),
                    text=['Amazon Headquarters'],
                ),
            ],
            'layout': go.Layout(
                title='Scatter Mapbox - Amazon Headquarters Location',
                mapbox=dict(
                    style="open-street-map",
                    center=dict(lat=37.7749, lon=-122.4194),
                    zoom=10,
                ),
                hovermode='closest',
            ),
        },
    ),

    # Donut Plot - Amazon Market Share
    dcc.Graph(
        id='donut-plot',
        figure={
            'data': [
                go.Pie(
                    labels=['Amazon', 'Other'],
                    values=[90, 10],
                    hole=0.4,
                ),
            ],
            'layout': go.Layout(
                title='Donut Plot - Amazon Market Share',
                hovermode='closest',
            ),
        },
    ),

    # Waterfall Chart
    dcc.Graph(
        id='waterfall-chart',
        figure={
            'data': [
                go.Waterfall(
                    x=amazon_stock_data.index,
                    y=amazon_stock_data['Close'].diff().fillna(0),
                    base=amazon_stock_data['Close'].iloc[0],
                    name='Waterfall Chart',
                ),
            ],
            'layout': go.Layout(
                title='Waterfall Chart - Daily Stock Price Changes',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Stock Price Change (USD)'},
                hovermode='closest',
            ),
        },
    ),

    # Parallel Coordinates Plot
    dcc.Graph(
        id='parallel-coordinates-plot',
        figure={
            'data': [
                go.Parcoords(
                    line=dict(color=amazon_stock_data['Close'], colorscale='Viridis'),
                    dimensions=[
                        dict(range=[min(amazon_stock_data['Open']), max(amazon_stock_data['Open'])],
                             label='Open', values=amazon_stock_data['Open']),
                        dict(range=[min(amazon_stock_data['High']), max(amazon_stock_data['High'])],
                             label='High', values=amazon_stock_data['High']),
                        dict(range=[min(amazon_stock_data['Low']), max(amazon_stock_data['Low'])],
                             label='Low', values=amazon_stock_data['Low']),
                        dict(range=[min(amazon_stock_data['Close']), max(amazon_stock_data['Close'])],
                             label='Close', values=amazon_stock_data['Close']),
                    ],
                ),
            ],
            'layout': go.Layout(
                title='Parallel Coordinates Plot - OHLC Values',
                hovermode='closest',
            ),
        },
    ),

    # Donut Chart - Distribution of Daily Stock Price Changes
    dcc.Graph(
        id='donut-chart-daily-changes',
        figure={
            'data': [
                go.Pie(
                    labels=['Increase', 'Decrease'],
                    values=[
                        len(amazon_stock_data[amazon_stock_data['Close'].diff().fillna(0) > 0]),
                        len(amazon_stock_data[amazon_stock_data['Close'].diff().fillna(0) <= 0]),
                    ],
                    hole=0.4,
                ),
            ],
            'layout': go.Layout(
                title='Donut Chart - Distribution of Daily Stock Price Changes',
                hovermode='closest',
            ),
        },
    ),
])

if __name__ == '__main__':
    app.run_server(debug=True)


[*********************100%%**********************]  1 of 1 completed


Mean Squared Error: 3.145658319905034e-20


<IPython.core.display.Javascript object>